In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [2]:
edu = pd.read_excel('Education.xlsx', header=3)
edu

,Federal Information Processing Standard (FIPS) Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2008-12","Percent of adults with a bachelor's degree or higher, 2008-12","Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21","Percent of adults with less than a high school diploma, 2017-21","Percent of adults with a high school diploma only, 2017-21","Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21"
0,0,US,United States,NaN,NaN,NaN,NaN,52373312.0,34158051.0,11650730.0,...,28.993579,28.484955,25050356.0,59636386.0,64656741.0,75808834.0,11.125960,26.487130,28.716889,33.670022
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,1062306.0,468269.0,136287.0,...,29.022866,22.264896,430047.0,1041209.0,1032122.0,910425.0,12.597300,30.499973,30.233789,26.668938
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6611.0,3757.0,933.0,...,29.618142,21.707831,4126.0,12977.0,11367.0,11144.0,10.415510,32.758621,28.694401,28.131469
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,18726.0,8426.0,2334.0,...,31.819503,27.741591,14555.0,44342.0,50518.0,52562.0,8.985844,27.375492,31.188379,32.450286
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,8120.0,2242.0,581.0,...,25.771784,14.524286,4378.0,6600.0,5010.0,2007.0,24.328980,36.676855,27.841067,11.153098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,72145,PR,Vega Baja Municipio,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,23.959976,17.584094,9466.0,11932.0,9246.0,8988.0,23.884740,30.106984,23.329633,22.678644
3281,72147,PR,Vieques Municipio,7.0,12.0,7.0,12.0,NaN,NaN,NaN,...,15.886045,10.140029,1613.0,2872.0,640.0,845.0,27.018425,48.107203,10.720268,14.154104
3282,72149,PR,Villalba Municipio,2.0,2.0,2.0,2.0,NaN,NaN,NaN,...,14.340816,15.207082,3329.0,5469.0,3392.0,3333.0,21.445597,35.231592,21.851446,21.471365
3283,72151,PR,Yabucoa Municipio,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,19.986177,12.912144,6097.0,6011.0,6631.0,3951.0,26.870868,26.491847,29.224328,17.412957


In [3]:
edu = edu[edu['State'] != 'PR']


In [4]:

edu = edu[['State', 'Area name', 'Percent of adults with less than a high school diploma, 2017-21', 'Percent of adults with a high school diploma only, 2017-21', 'Percent of adults completing some college or associate\'s degree, 2017-21', 'Percent of adults with a bachelor\'s degree or higher, 2017-21']]
edu.columns = ['State', 'county_name', 'noHS', 'HS', 'someCol', 'Col']
edu

,State,county_name,noHS,HS,someCol,Col
0,US,United States,11.125960,26.487130,28.716889,33.670022
1,AL,Alabama,12.597300,30.499973,30.233789,26.668938
2,AL,Autauga County,10.415510,32.758621,28.694401,28.131469
3,AL,Baldwin County,8.985844,27.375492,31.188379,32.450286
4,AL,Barbour County,24.328980,36.676855,27.841067,11.153098
...,...,...,...,...,...,...
3201,WY,Sweetwater County,7.294363,32.603538,40.710383,19.391717
3202,WY,Teton County,3.969647,16.897899,21.263945,57.868509
3203,WY,Uinta County,6.430892,36.424091,39.189904,17.955112
3204,WY,Washakie County,5.900793,25.650009,42.282869,26.166329


In [5]:
#edu.to_csv('edu.csv')
#edu

In [6]:
kag = pd.read_csv('cost_of_living_us.csv')
kag

,case_id,state,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,healthcare_cost,other_necessities_cost,childcare_cost,taxes,total_cost,median_family_income
0,1,AL,True,"Montgomery, AL MSA",Autauga County,1p0c,8505.72876,3454.91712,10829.16876,5737.47984,4333.81344,0.0000,6392.94504,39254.0532,73010.414062
1,1,AL,True,"Montgomery, AL MSA",Autauga County,1p1c,12067.50240,5091.70788,11588.19288,8659.55640,6217.45896,6147.8298,7422.07836,57194.3256,73010.414062
2,1,AL,True,"Montgomery, AL MSA",Autauga County,1p2c,12067.50240,7460.20308,12361.77720,11581.63260,7075.65816,15824.6940,9769.56228,76141.0308,73010.414062
3,1,AL,True,"Montgomery, AL MSA",Autauga County,1p3c,15257.15040,9952.23924,13452.18600,14503.70760,9134.35620,18802.1892,13101.70320,94203.5328,73010.414062
4,1,AL,True,"Montgomery, AL MSA",Autauga County,1p4c,15257.15040,12182.21400,13744.59840,17425.78560,9942.36396,18802.1892,13469.21880,100823.5200,73010.414062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31425,3171,WY,False,"Weston County, WY",Weston County,2p0c,8316.00000,6010.65672,14790.36240,14466.55920,5191.11288,0.0000,6640.77324,55415.4672,80177.656250
31426,3171,WY,False,"Weston County, WY",Weston County,2p1c,10956.00000,7480.86696,15611.58600,18086.24160,6680.40384,8675.8770,7933.21068,75424.1832,80177.656250
31427,3171,WY,False,"Weston County, WY",Weston County,2p2c,10956.00000,9474.35748,16163.90280,21705.92280,7402.72416,21207.6984,9502.56228,96413.1684,80177.656250
31428,3171,WY,False,"Weston County, WY",Weston County,2p3c,13632.00000,11567.46972,17118.38400,25325.60400,9130.76148,25063.6440,11456.36724,113294.2260,80177.656250


In [7]:
#state_abv = []
#state_abv.append([re.findall('(, [A-Z]{2})', str(x)) for x in kag['areaname']][0])
#state_abv.append([re.findall('(, [A-Z]{2})', str(x)) for x in kag['areaname']][1])

In [8]:
#state_abv
#range(len(kag))

In [9]:
#state_abv = []
#for i in range(1):
#    state_abv.append([re.findall('(, [A-Z]{2})', str(x)) for x in kag['areaname']][i])

In [10]:
#state_abv

In [11]:
#kag_piv = pd.pivot_table(kag, index=["county", 'state', 'median_family_income'], columns="family_member_count", values=["total_cost"])

In [12]:
#kag_piv.droplevel(0, axis=1).reset_index()


In [13]:
merge1 = pd.merge(kag, edu, left_on=['county', 'state'], right_on=['county_name', 'State'], how='inner')
merge1 = merge1.drop(columns=['State', 'county_name'])
merge1.county = merge1.county.str.replace(' County', ' ')
merge1.county = merge1.county.str.replace(' Parish', ' ')
merge1.county = merge1.county.str.strip()
merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31420 entries, 0 to 31419
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_id                 31420 non-null  int64  
 1   state                   31420 non-null  object 
 2   isMetro                 31420 non-null  bool   
 3   areaname                31420 non-null  object 
 4   county                  31420 non-null  object 
 5   family_member_count     31420 non-null  object 
 6   housing_cost            31420 non-null  float64
 7   food_cost               31420 non-null  float64
 8   transportation_cost     31420 non-null  float64
 9   healthcare_cost         31420 non-null  float64
 10  other_necessities_cost  31420 non-null  float64
 11  childcare_cost          31420 non-null  float64
 12  taxes                   31420 non-null  float64
 13  total_cost              31420 non-null  float64
 14  median_family_income    31420 non-null

In [14]:
abs = pd.read_csv('StateAbbvs.csv', header=None)
abs.columns = ['State', 'Abv']
abs.State = abs.State.str.upper()
abs

,State,Abv
0,ALABAMA,AL
1,ALASKA,AK
2,ARIZONA,AZ
3,ARKANSAS,AR
4,CALIFORNIA,CA
5,CANAL ZONE,CZ
6,COLORADO,CO
7,CONNECTICUT,CT
8,DELAWARE,DE
9,DISTRICT OF COLUMBIA,DC


In [15]:
vcounty= pd.read_csv('vcounty.csv')
vcounty = vcounty[['state', 'county_name', 'totalvotes', 'DEMOCRAT', 'GREEN', 'LIBERTARIAN', 'OTHER', 'REPUBLICAN']]
vcounty

,state,county_name,totalvotes,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN
0,ALABAMA,Autauga,27770,0.270184,0.0,0.000000,0.015448,0.714368
1,ALABAMA,Baldwin,109679,0.224090,0.0,0.000000,0.014196,0.761714
2,ALABAMA,Barbour,10518,0.457882,0.0,0.000000,0.007606,0.534512
3,ALABAMA,Bibb,9595,0.206983,0.0,0.000000,0.008755,0.784263
4,ALABAMA,Blount,27588,0.095694,0.0,0.000000,0.008591,0.895716
...,...,...,...,...,...,...,...,...
2301,WYOMING,Sweetwater,16698,0.228950,0.0,0.020961,0.017727,0.732363
2302,WYOMING,Teton,14787,0.665990,0.0,0.017245,0.023196,0.293569
2303,WYOMING,Uinta,9459,0.168200,0.0,0.018184,0.021144,0.792473
2304,WYOMING,Washakie,4032,0.161458,0.0,0.016121,0.017609,0.804812


In [16]:
vcounty = pd.merge(vcounty, abs, left_on = 'state', right_on = 'State', how='inner')
vcounty = vcounty.drop(columns=['State'])

vcounty.Abv = vcounty.Abv.str.strip()
vcounty.county_name = vcounty.county_name.str.strip()
vcounty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 0 to 2305
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   state        2306 non-null   object 
 1   county_name  2306 non-null   object 
 2   totalvotes   2306 non-null   int64  
 3   DEMOCRAT     2306 non-null   float64
 4   GREEN        2306 non-null   float64
 5   LIBERTARIAN  2306 non-null   float64
 6   OTHER        2306 non-null   float64
 7   REPUBLICAN   2306 non-null   float64
 8   Abv          2306 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 180.2+ KB


In [17]:
final = pd.merge(merge1, vcounty, left_on=['state', 'county'], right_on = ['Abv', 'county_name'], how='inner')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22190 entries, 0 to 22189
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_id                 22190 non-null  int64  
 1   state_x                 22190 non-null  object 
 2   isMetro                 22190 non-null  bool   
 3   areaname                22190 non-null  object 
 4   county                  22190 non-null  object 
 5   family_member_count     22190 non-null  object 
 6   housing_cost            22190 non-null  float64
 7   food_cost               22190 non-null  float64
 8   transportation_cost     22190 non-null  float64
 9   healthcare_cost         22190 non-null  float64
 10  other_necessities_cost  22190 non-null  float64
 11  childcare_cost          22190 non-null  float64
 12  taxes                   22190 non-null  float64
 13  total_cost              22190 non-null  float64
 14  median_family_income    22190 non-null

In [18]:
#final
final.to_csv('countyVLivEdu.csv')